In [ ]:
8192**(1/3)

20.158736798317967

In [ ]:
8192**(1/2)

90.50966799187809

In [ ]:
8192/2/2

2048.0

In [ ]:
from math import *
log2(8192)

13.0

In [ ]:
2**13 

8192

## run the lab

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [2]:
file_path = "/content/drive/MyDrive/Fourier-Transformer/FNO data/data/burgers_data_R10.mat"

In [7]:
"""
@author: Zongyi Li
This file is the Fourier Neural Operator for 1D problem such as the (time-independent) Burgers equation discussed in Section 5.1 in the [paper](https://arxiv.org/pdf/2010.08895.pdf).
"""

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

# import matplotlib  
# matplotlib.use('TkAgg')   
import matplotlib.pyplot as plt  

import operator
from functools import reduce
from functools import partial
from timeit import default_timer


torch.manual_seed(0)
np.random.seed(0)

In [19]:
%cd "/content/drive/MyDrive/GitHub/Fourier-Transformer/fourier_neural_operator"

/content/drive/MyDrive/GitHub/Fourier-Transformer/fourier_neural_operator


In [20]:
!pwd

/content/drive/MyDrive/GitHub/Fourier-Transformer/fourier_neural_operator


In [21]:
from utilities3 import *

In [22]:
from Adam import Adam

In [23]:
################################################################
#  1d fourier layer
################################################################
class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  #Number of Fourier modes to multiply, at most floor(N/2) + 1

        self.scale = (1 / (in_channels*out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul1d(self, input, weights):
        # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-1)//2 + 1,  device=x.device, dtype=torch.cfloat)
        out_ft[:, :, :self.modes1] = self.compl_mul1d(x_ft[:, :, :self.modes1], self.weights1)

        #Return to physical space
        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x

class FNO1d(nn.Module):
    def __init__(self, modes, width):
        super(FNO1d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .
        
        input: the solution of the initial condition and location (a(x), x)
        input shape: (batchsize, x=s, c=2)
        output: the solution of a later timestep
        output shape: (batchsize, x=s, c=1)
        """

        self.modes1 = modes
        self.width = width
        self.padding = 2 # pad the domain if input is non-periodic
        self.fc0 = nn.Linear(2, self.width) # input channel is 2: (a(x), x)

        self.conv0 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv1 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv2 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv3 = SpectralConv1d(self.width, self.width, self.modes1)
        self.w0 = nn.Conv1d(self.width, self.width, 1)
        self.w1 = nn.Conv1d(self.width, self.width, 1)
        self.w2 = nn.Conv1d(self.width, self.width, 1)
        self.w3 = nn.Conv1d(self.width, self.width, 1)

        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 2, 1)
        # x = F.pad(x, [0,self.padding]) # pad the domain if input is non-periodic

        x1 = self.conv0(x)
        x2 = self.w0(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv1(x)
        x2 = self.w1(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv2(x)
        x2 = self.w2(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv3(x)
        x2 = self.w3(x)
        x = x1 + x2

        # x = x[..., :-self.padding] # pad the domain if input is non-periodic
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)
        return x

    def get_grid(self, shape, device):
        batchsize, size_x = shape[0], shape[1]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1).repeat([batchsize, 1, 1])
        return gridx.to(device)



In [25]:
################################################################
#  configurations
################################################################
ntrain = 1000
ntest = 100

sub = 2**3 #subsampling rate
h = 2**13 // sub #total grid size divided by the subsampling rate
s = h

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 50
gamma = 0.5

modes = 16
width = 64


################################################################
# read data
################################################################

# Data is of the shape (number of samples, grid size)
# dataloader = MatReader('data/burgers_data_R10.mat')
dataloader = MatReader(file_path)
x_data = dataloader.read_field('a')[:,::sub]
y_data = dataloader.read_field('u')[:,::sub]

x_train = x_data[:ntrain,:]
y_train = y_data[:ntrain,:]
x_test = x_data[-ntest:,:]
y_test = y_data[-ntest:,:]

x_train = x_train.reshape(ntrain,s,1)
x_test = x_test.reshape(ntest,s,1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)



In [26]:
# model
model = FNO1d(modes, width).cuda()
print(count_params(model))



549569


In [27]:
################################################################
# training and evaluation
################################################################
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_mse = 0
    train_l2 = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out = model(x)

        mse = F.mse_loss(out.view(batch_size, -1), y.view(batch_size, -1), reduction='mean')
        l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1))
        l2.backward() # use the l2 relative loss

        optimizer.step()
        train_mse += mse.item()
        train_l2 += l2.item()

    scheduler.step()
    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()

            out = model(x)
            test_l2 += myloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()

    train_mse /= len(train_loader)
    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_mse, train_l2, test_l2)



0 1.1458250420000695 0.0565736242569983 0.33394574308395386 0.06045842528343201
1 0.6263968069999919 0.001110132631729357 0.04657960855960846 0.04164898753166199
2 0.6263320230000318 0.0005877650128968526 0.0343597574532032 0.021544314622879028
3 0.6236622559999887 0.0003308755253237905 0.023728334903717042 0.05330180168151855
4 0.623827408000011 0.0002806643042276846 0.024962983697652815 0.02260588973760605
5 0.6306298250000282 0.0003597614153113682 0.03258722087740898 0.04092469334602356
6 0.6290781960000231 0.00035832139023114 0.03183681207895279 0.04338636815547943
7 0.6262231609999844 0.00016212167603953275 0.01943308261036873 0.020751235485076906
8 0.6258100739999009 0.0001707980587525526 0.022373816788196565 0.02758720636367798
9 0.6280023770000298 0.00021239155990770086 0.02484304466843605 0.03177709639072418
10 0.6248597850000124 0.00014185922173055588 0.020493721187114716 0.01725591391324997
11 0.6246402310000576 0.00017228139055077918 0.023372262373566627 0.04448602318763733

In [28]:
# torch.save(model, 'model/ns_fourier_burgers')
pred = torch.zeros(y_test.shape)
index = 0
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=1, shuffle=False)
with torch.no_grad():
    for x, y in test_loader:
        test_l2 = 0
        x, y = x.cuda(), y.cuda()

        out = model(x).view(-1)
        pred[index] = out

        test_l2 += myloss(out.view(1, -1), y.view(1, -1)).item()
        print(index, test_l2)
        index = index + 1

# scipy.io.savemat('pred/burger_test.mat', mdict={'pred': pred.cpu().numpy()})

0 0.00020691305689979345
1 0.0007236276287585497
2 0.00021344282140489668
3 0.0003020109434146434
4 0.00017791384016163647
5 0.0003393020306248218
6 0.00036226652446202934
7 0.00028611207380890846
8 0.00021226641547400504
9 0.0003139404288958758
10 0.0005580832366831601
11 0.00028766761533915997
12 0.0003050846862606704
13 0.00047370174434036016
14 0.0002041904954239726
15 0.00046032306272536516
16 0.00018010643543675542
17 0.0003812407376244664
18 0.00041470519499853253
19 0.0007646279991604388
20 0.0003480752056930214
21 0.0004061970394104719
22 0.0006313658668659627
23 0.0012994724093005061
24 0.00018685169925447553
25 0.0005111137288622558
26 0.00038473133463412523
27 0.00021119200391694903
28 0.0005009156302548945
29 0.000645210559014231
30 0.00040109537076205015
31 0.00042046065209433436
32 0.0009987801313400269
33 0.0005689954268746078
34 0.0013601185055449605
35 0.00046117062447592616
36 0.00022893732239026576
37 0.0004971221787855029
38 0.00018867682956624776
39 0.000251790013